In [125]:
import pandas as pd


def player_per90_percentiles(df, per90=True, percentile=True):
    semantic_cols = ["ranker", "player", "nationality", "position", "team", "age", "birth_year", "games", "games_starts", "minutes", "minutes_90s"]

    stats_cols = df.select_dtypes(include="number").columns
    cols_to_transform = [c for c in stats_cols if c not in semantic_cols]

    res_df = df[cols_to_transform].copy()

    # ---- Per 90 ----
    if per90:
        base_cols = [c for c in res_df.columns if not c.endswith("_per90")]
        res_df[base_cols] = res_df[base_cols].div(df["minutes_90s"], axis=0)
        res_df.rename(columns={c: f"{c}_per90" for c in base_cols}, inplace=True)

    res_df = res_df.loc[:, ~res_df.columns.duplicated()]
    # ---- Percentiles ----
    if percentile:
        pct_base_cols = [c for c in res_df.columns if not c.endswith("%")]

        ranked = res_df[pct_base_cols].rank(pct=True)
        ranked = ranked.add_suffix("%")

        # keep already-percent columns + new percentiles
        res_df = pd.concat([res_df.drop(columns=pct_base_cols), ranked], axis=1)

    final_df = pd.concat([df[["player", "position", "team"]], res_df], axis=1)

    return final_df


In [126]:
import plotly.graph_objects as go


def plot_plotly_radar_compare(df, player_names, params):
    fig = go.Figure()

    for player_name in player_names:
        player_row = df[df["player"] == player_name].iloc[0]

        values = [player_row[p] if p.endswith("%") else player_row[p] * 100 for p in params]
        values.append(values[0])
        l_params = params + [params[0]]

        fig.add_trace(go.Scatterpolar(r=values, theta=l_params, fill="toself", name=player_name, opacity=0.45))

    fig.update_layout(
        polar=dict(radialaxis=dict(range=[0, 100], gridcolor="lightgrey"), angularaxis=dict(rotation=90, direction="clockwise")),
        title="Player Radar Comparison",
        template="plotly_white",
        showlegend=True,
    )

    return fig  # ⬅️ RETURN, do not show


In [127]:
from pathlib import Path

root = Path.cwd()
while not (root / ".git").exists() and root.parent != root:
    root = root.parent

DATA_PATH = root / "data" / "fbref"
year = "25_26"
prefix = "stats"
df = pd.read_json(DATA_PATH / "PL_outfield" / year / f"PL_outfield_{year}_{prefix}.json")

semantic_cols = set(["ranker", "player", "nationality", "position", "team", "age", "birth_year", "games", "games_starts", "minutes"])
team_semantic_cols = set(["team", "players_used", "avg_age", "games", "games_starts", "minutes"])

params = list(set(df.columns.to_list()) - semantic_cols - team_semantic_cols)

params

['goals_pens_per90',
 'npxg',
 'npxg_xg_assist_per90',
 'progressive_carries',
 'xg_per90',
 'assists',
 'npxg_per90',
 'assists_per90',
 'xg_xg_assist_per90',
 'cards_red',
 'xg_assist',
 'progressive_passes',
 'goals',
 'xg',
 'npxg_xg_assist',
 'minutes_90s',
 'goals_pens',
 'cards_yellow',
 'goals_assists',
 'goals_assists_per90',
 'xg_assist_per90',
 'goals_assists_pens_per90',
 'goals_per90',
 'progressive_passes_received',
 'pens_att',
 'pens_made']

In [128]:
new_df = player_per90_percentiles(df)
print("all new processed dataframe columns")
params = sorted(new_df.columns.to_list())
params

all new processed dataframe columns


['assists_per90%',
 'cards_red_per90%',
 'cards_yellow_per90%',
 'goals_assists_pens_per90%',
 'goals_assists_per90%',
 'goals_pens_per90%',
 'goals_per90%',
 'npxg_per90%',
 'npxg_xg_assist_per90%',
 'pens_att_per90%',
 'pens_made_per90%',
 'player',
 'position',
 'progressive_carries_per90%',
 'progressive_passes_per90%',
 'progressive_passes_received_per90%',
 'team',
 'xg_assist_per90%',
 'xg_per90%',
 'xg_xg_assist_per90%']

In [129]:
import streamlit as st

st.title("Player Radar Comparison")

players = st.multiselect("Players", new_df["player"].unique(), default=list(new_df["player"].unique()[:2]))

metrics = st.multiselect("Metrics", params, default=params)

if len(players) < 2:
    st.info("Select at least two players.")
elif not metrics:
    st.info("Select at least one metric.")
else:
    fig = plot_plotly_radar_compare(new_df, players, metrics)
    st.plotly_chart(fig, width="stretch")


2026-01-25 00:11:08.899 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-25 00:11:08.901 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-25 00:11:08.902 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-25 00:11:08.905 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-25 00:11:08.907 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-25 00:11:08.909 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-25 00:11:08.911 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-25 00:11:08.912 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar